In [153]:
import numpy as np

# finds the intersection point between the quadric 
# <x|Qx> + <b|x> + c = 0 (Q symmetric) and the line t -> a + tv
# i.e finds t such that 
# <a+tv|Q(a+tv)> + <b|a+tv> + c = 0
# <=> <a|Qa> + 2t<a|Qv> + t^2 <v|Qv> + <b|a> + t<b|v> + c = 0
# <=> A t^2 + B t + C = 0
# with A = <v|Qv>, B = 2<a|Qv> + <b|v>, C = c + <b|a> + <a|Qa> 

def ray_quadric_intersection(Q, b, c, a, v):
    A = np.dot(v,Q@v)
    B = 2*np.dot(a,Q@v) + np.dot(b,v)
    C = c + np.dot(b,a) + np.dot(a,Q@a)
    
    # linear case
    if np.abs(A) < 1e-7:
        print("linear case")
        if np.abs(B) > 1e-7:
            return -C/B
        else:
            return
        
    Delta = B**2 - 4*A*C
    if Delta <= 0:
        #print("negative Delta={}".format(Delta))
        #print("a={}, v={}".format(a,v))
        return
    sqrtDelta = np.sqrt(Delta)
    return (-B+sqrtDelta)/(2*A), (-B-sqrtDelta)/(2*A)

def segment_quadric_intersection(Q, b, c, p, q):
    T = ray_quadric_intersection(Q, b, c, p, q-p)
    if T == None:
        return None
    T = [t for t in T if (t >= 0) and t <= 1]
    return T

def intermediate_point_on_quadric(Q, b, c, p, q, s, norm):
    v = np.cross(q-p, norm)
    print(v)
    #if v[2] > 0:
    #    v = -v
    a = (1-s)*p+s*q
    T = ray_quadric_intersection(Q, b, c, a, v)
    T = [t for t in T if t >= 0 and (a+t*v)[2] >=0]
    print(len(T))
    assert(len(T) == 1)
    return a + T[0]*v
    
def trace_curve_on_quadric(Q,b,c,p,q,norm,N):
    S = np.linspace(0,1,N)[1:-1]
    return [p]+[intermediate_point_on_quadric(Q,b,c,p,q,s,norm) for s in S]+[q]

In [2]:
# quadric parametrized by X -> sqrt(|X-p|^2 + beta) + b_i
# <=> z = sqrt((x-px)^2 + (y-py)^2 + beta) + b_i
# <=> (z-b_i)^2 = (x-px)^2 + (y-py)^2 + beta
# <=> <Q(X-R)|(X-R)> + beta = 0
# with Q = diag(1,1,-1), R = (px,py,bi)
# <=> <QX|X> + 2<QR|X> + <QR|R> + beta
Q = np.diag([1,1,-1])
R = np.zeros(3)
beta = 1.
b = 2*Q@R
c = np.dot(Q@R,R) + beta

def quad(x):
    return np.dot(x,Q@x) + np.dot(b,x) + c

# test 0
a = np.array([0,0,3])
v = np.random.rand(3)
T = ray_quadric_intersection(Q,b,c,a,v)
for t in T:
    print(quad(a+t*v))
    
# test 1
p = np.array([0,0,3])
q = np.random.rand(3)
q[2] = 0
T = segment_quadric_intersection(Q,b,c,p,q)
for t in T:
    print(quad(p+t*(q-p)))

# test 2
q = np.array([1.,0,0])
p = np.array([0.,1,0])
q[2] = np.sqrt(q[0]**2 + q[1]**2 + beta)
p[2] = np.sqrt(p[0]**2 + p[1]**2 + beta)
print("quad(p) = {}, quad(q) = {}".format(quad(p),quad(q)))

#norm = np.random.rand(3)
norm = np.array([0,0,-1])
# on veut p-q \in norm^\perp
# soit <p-q|norm> = 0
norm = norm - np.dot(p-q,norm)*(p-q)/np.linalg.norm(p-q)**2
print("n:", np.dot(p-q,norm))
print(norm)

curve = trace_curve_on_quadric(Q,b,c,p,q,norm,10)
for x in curve:
    print(x)
    print("{} {} {}".format(quad(x),np.dot(x-p,norm),np.dot(x-q,norm)))

    
import matplotlib.pyplot as plt
curve = np.array(curve)
plt.axis('equal')
plt.plot(curve[:,0], curve[:,1])

-3.552713678800501e-15
4.440892098500626e-16
-1.1102230246251565e-15
quad(p) = -4.440892098500626e-16, quad(q) = -4.440892098500626e-16
n: 0.0
[ 0.  0. -1.]
1
1
1
1
1
1
1
1
[0.         1.         1.41421356]
-4.440892098500626e-16 0.0 0.0
[0.20167477 0.97945255 1.41421356]
0.0 2.220446049250313e-16 2.220446049250313e-16
[0.37248333 0.92803888 1.41421356]
0.0 0.0 0.0
[0.5205176  0.85385094 1.41421356]
2.220446049250313e-16 -2.220446049250313e-16 -2.220446049250313e-16
[0.64936542 0.76047653 1.41421356]
0.0 0.0 0.0
[0.76047653 0.64936542 1.41421356]
0.0 0.0 0.0
[0.85385094 0.5205176  1.41421356]
-2.220446049250313e-16 0.0 0.0
[0.92803888 0.37248333 1.41421356]
2.220446049250313e-16 0.0 0.0
[0.97945255 0.20167477 1.41421356]
0.0 2.220446049250313e-16 2.220446049250313e-16
[1.         0.         1.41421356]
-4.440892098500626e-16 0.0 0.0


In [4]:
import pypower

In [10]:
X = np.random.rand(10,3)
w = np.zeros(10)
cells = pypower.power_diagram(X,w)

In [16]:
for a in cells[0]:
    print("numéro", a)
    print("facette", cells[0][a])

numéro -5
facette [array([ 1.        , -0.57199131,  1.        ]), array([ 1., -1.,  1.]), array([ 1.        , -1.        ,  0.21991935]), array([1.        , 0.02974732, 0.59638845])]
numéro -4
facette [array([-0.79461892, -1.        ,  1.        ]), array([ 0.51120221, -1.        , -0.03717408]), array([ 1.        , -1.        ,  0.21991935]), array([ 1., -1.,  1.])]
numéro -1
facette [array([-0.25167222,  0.29891164,  1.        ]), array([-0.79461892, -1.        ,  1.        ]), array([ 1., -1.,  1.]), array([ 1.        , -0.57199131,  1.        ])]
numéro 1
facette [array([0.50763271, 0.50136766, 0.50983898]), array([0.50332455, 0.50005735, 0.50709398]), array([0.43650233, 0.49396651, 0.54461701]), array([0.43433041, 0.49429397, 0.54879356])]
numéro 3
facette [array([0.50332455, 0.50005735, 0.50709398]), array([0.4741737 , 0.4554781 , 0.47546359]), array([0.36785515, 0.47226625, 0.56548294]), array([0.43650233, 0.49396651, 0.54461701])]
numéro 4
facette [array([-0.25167222,  0.29891

In [19]:
import mpl_toolkits.mplot3d as a3
import matplotlib.colors as colors
import pylab as pl
import scipy as sp

ax = a3.Axes3D(pl.figure())
for c in cells:
    for k,vtx in c.items():
        print(k)
        #tri = a3.art3d.Poly3DCollection([vtx])
        #tri.set_color(colors.rgb2hex(sp.rand(3)))
        #tri.set_alpha(0.1)
        #tri.set_edgecolor('k')
        #ax.add_collection3d(tri)
        vtx = np.array(vtx)
        ax.plot3D(vtx[:,0], vtx[:,1], vtx[:,2])
ax.autoscale()

-5
-4
-1
1
3
4
5
6
7
-3
-1
0
2
3
4
5
7
8
9
-5
-3
-2
1
7
8
9
-6
-3
-2
0
1
5
6
7
8
9
-5
-3
-1
0
1
5
6
7
9
-6
-3
-1
0
1
3
4
6
8
-6
-4
-2
-1
0
3
4
5
7
-5
-4
-2
0
1
2
3
4
6
8
9
-6
-3
-2
1
2
3
5
7
9
-5
-3
-2
1
2
3
4
7
8


In [18]:
%matplotlib qt

In [182]:
Q = np.diag([-1,-1,1])
b = np.zeros(3)
b[2] = 1
c = 0

def quad(x):
    return np.dot(x,Q@x) + np.dot(b,x) + c

def ray_quadric_first_intersection(Q, b, c, a, v):
    S = ray_quadric_intersection(Q, b, c, a, v)
    S = np.sort(S)
    S = S[S>= -1e-5]
    return a + S[0] * v


def intermediate_point_on_quadric2(Q, b, c, p, q, s, bary):
    a = (1-s)*p+s*q
    print("qa=%g, qb =%g"%(quad(a), quad(bary)))
    return ray_quadric_first_intersection(Q, b, c, a, bary-a)

    
def trace_curve_on_quadric2(Q,b,c,p,q,bary,N):
    p = p + 0.1*(p-bary)
    q = q + 0.1*(q-bary)
    S = np.linspace(0,1,N)[1:-1]
    return [p]+[intermediate_point_on_quadric2(Q,b,c,p,q,s,bary) for s in S]+[q]

def intersect_cell_with_quadric(X, cells, i, N = 10):
    curves = []
    cell = cells[i]
    for j,P in cell.items(): # iterate over all facets
        nv = len(P)
        P = np.array(P)
        sign = np.sign([quad(x) for x in P])
        norm = X[j,:] - X[i,:]
        
        # find point above quadric in P
        above = None
        for v in range(nv):
            if sign[v] > 0:
                above = P[v,:] + 1e-5*(np.mean(P,0) - P[v,:])
                break
        if above is None:
            for v in range(nv):
                w = (v+1)%nv
                T = segment_quadric_intersection(Q, b, c, P[v], P[w])
                if T == None or T == []:
                    continue
                assert(len(T) == 2)
                q1 = P[v] + T[0]*(P[w] - P[v])
                q2 = P[v] + T[1]*(P[w] - P[v])
                above = (q1+q2)/2 + 1e-7*(np.mean(P,0) - P[v,:])
                print("here", quad(above))
                break
                
        if above is None:
            return None
        
        bary = above
        print(bary)
        print(quad(bary))
        
        assert(np.sign(quad(bary)) > 0)
        
        # iterate over the vertices of the facet adjacent to j
        for v in range(nv):
            w = (v+1)%nv
            if (sign[v] == 1) and (sign[w] == 1): # all above: nothing to do
                continue
            elif (sign[v] == 1) and (sign[w] == -1): 
                T = segment_quadric_intersection(Q, b, c, P[v], P[w])
                assert len(T) == 1
                p = P[v] + T[0]*(P[w] - P[v])
                q = ray_quadric_first_intersection(Q, b, c, P[w], bary-P[w])
                curves.append(trace_curve_on_quadric2(Q,b,c,p,q,bary,N))
            elif (sign[v] == -1) and (sign[w] == 1): 
                p = ray_quadric_first_intersection(Q, b, c, P[v], bary-P[v])
                T = segment_quadric_intersection(Q, b, c, P[v], P[w])
                assert(len(T) == 1)
                q = P[v] + T[0]*(P[w] - P[v])
                curves.append(trace_curve_on_quadric2(Q,b,c,p,q,bary,N))
            elif (sign[v] == -1) and (sign[w] == -1): 
                pv = ray_quadric_first_intersection(Q, b, c, P[v], bary-P[v])
                pw = ray_quadric_first_intersection(Q, b, c, P[w], bary-P[w])
                T = segment_quadric_intersection(Q, b, c, P[v], P[w])
                if T == None or T == []:
                    continue
                assert(len(T) == 2)
                q1 = P[v] + T[0]*(P[w] - P[v])
                q2 = P[v] + T[1]*(P[w] - P[v])
                curves.append(trace_curve_on_quadric2(Q,b,c,pv,q1,bary,N))
                curves.append(trace_curve_on_quadric2(Q,b,c,q2,pw,bary,N))
    return curves
            
X = np.random.rand(10,3)
w = np.zeros(10)
cells = pypower.power_diagram(X,w)
for i in range(10):
    curves = intersect_cell_with_quadric(X,cells,i)
    if curves is None:
        continue
    for crv in curves:
        crv = np.array(crv)
        plt.plot(crv[:,0], crv[:,1])
    break

[1.         0.80622043 0.9999977 ]
0.35000172268198815
qa=-0.0334161, qb =0.350002
qa=-0.0334161, qb =0.350002
qa=-0.0334161, qb =0.350002
qa=-0.0334161, qb =0.350002
qa=-0.033416, qb =0.350002
qa=-0.033416, qb =0.350002
qa=-0.033416, qb =0.350002
qa=-0.033416, qb =0.350002
qa=-0.0512139, qb =0.350002
qa=-0.0281104, qb =0.350002
qa=-0.0110645, qb =0.350002
qa=-7.62744e-05, qb =0.350002
qa=0.00485439, qb =0.350002
qa=0.00372744, qb =0.350002
qa=-0.00345712, qb =0.350002
qa=-0.0166993, qb =0.350002
[0.99999644 0.80622193 1.        ]
0.35001332678106545
[0.27002328 0.45757588 1.        ]
1.7177117417522265
[1.         0.80622625 0.99999704]
0.3499903639379649
qa=-0.0334152, qb =0.34999
qa=-0.0334152, qb =0.34999
qa=-0.0334152, qb =0.34999
qa=-0.0334152, qb =0.34999
qa=-0.0334151, qb =0.34999
qa=-0.0334151, qb =0.34999
qa=-0.0334151, qb =0.34999
qa=-0.0334151, qb =0.34999
[-0.96312815  1.          0.99999544]
0.07237048791006029
qa=-0.00714382, qb =0.0723705
qa=-0.00714382, qb =0.0723705
q

AssertionError: 

In [105]:
import matplotlib.pyplot as plt
%matplotlib inline



In [106]:
print(crv)

[[0.07052921 0.95163309 0.57730214]
 [0.18895212 0.28511546 0.10580007]
 [0.19064835 0.28548794 0.10650652]
 [0.19232822 0.28587135 0.10721708]
 [0.19399219 0.28626534 0.10793159]
 [0.19564067 0.28666958 0.10864992]
 [0.19727408 0.28708374 0.10937192]
 [0.19889281 0.28750753 0.11009748]
 [0.20049723 0.28794066 0.11082646]
 [0.10892604 1.         0.62332759]]
